# Python Client API for WFS and WTSS

## Building and installing simple_geo.py from source

1. In the shell, type

```bash


  git clone https://github.com/vconrado/simple_geo.py.git
  cd simple_geo.py/src
  pip install .
```

## Using SimpleGeo.py

### Listing Features and Coverages

In the following example, we import the SimpleGeo module and then create a SimpleGeo object to query and print the list of available features and coverages in the server.


In [1]:
from SimpleGeo import SimpleGeo

# connect to wfs and wtss servers
s = SimpleGeo(wfs="http://wfs_server:8080/geoserver-esensing", wtss="http://wtss_server:7654")

# print available features
print(s.features())

# print available coverages
print(s.coverages())

{u'features': [u'esensing:estados_bra', u'esensing:focos_bra_2016', u'esensing:focos_bra_2016_01_01', u'esensing:municipios_bra']}
{u'coverages': [u'climatologia', u'landcover_igbp_2012', u'ocorrencia_focos_2016', u'rpth']}


## Getting metadata

In the following example, we use SimpleGeo to retrieve the details of a feature and a coverage. Then we format the response.

In [2]:
import json

# Feature estados_bra
f = s.feature('esensing:estados_bra')
print(f.describe())


# Coverage rpth
c = s.coverage('rpth')
print(c.describe())

{'geometry': {'localtype': u'MultiPolygon', 'type': u'gml:MultiPolygon', 'name': u'geometria'}, 'attributes': [{'localtype': u'string', 'type': u'xsd:string', 'name': u'nome'}, {'localtype': u'MultiPolygon', 'type': u'gml:MultiPolygon', 'name': u'geometria'}, {'localtype': u'string', 'type': u'xsd:string', 'name': u'pais'}, {'localtype': u'string', 'type': u'xsd:string', 'name': u'regiao'}], 'namespace': u'esensing', 'name': u'estados_bra', 'full_name': 'esensing:estados_bra'}
{u'crs': {u'proj4': u'+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', u'wkt': u'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'}, u'dimensions': {u'y': {u'max_idx': 1399, u'min_idx': 0, u'name': u'row_id'}, u'x': {u'max_idx': 1199, u'min_idx': 0, u'name': u'col_id'}, u't': {u'max_idx': 365, u'min_idx': 

## Retrieving features

In the following example, we retrieve a feature from the WFS server. SimpleGeo returns a GeoPandasDataFrame with the features.

In [6]:
f = s.feature('esensing:estados_bra')
estados = f.get()
# displaying 
display(estados.head())

,geometry,nome,pais,regiao
0,"(POLYGON ((-37.99244578 -9.52510618, -37.98933...",Sergipe,Brazil,NE
1,"(POLYGON ((-37.1500849 -4.93569494, -37.147238...",Rio Grande do Norte,Brazil,NE
2,"(POLYGON ((-41.74604645 -2.8034972, -41.742407...",Piauí,Brazil,NE
3,"(POLYGON ((-48.35878217 -5.17008472, -48.35617...",Tocantins,Brazil,N
4,"(POLYGON ((-60.20050658 5.26434344, -60.198284...",Roraima,Brazil,N


### Feature methods

- **max_features**: limit the number of features returned
- **sort_by**: sorts by the returning features using one (ou more) attributes
- **attributes**: selects attributes to be retrieved
- **filter**: filtering features by its spatial and non-spatial attributes


In the following example, we retrieve a feature from the WFS server using all allowed options. You can combine then in many ways to select only the features you want.

In [7]:
# import SimpleGeo Predicates
from SimpleGeo import Predicates as pre

f = s.feature("esensing:focos_bra_2016") \
    .attributes(["id", "municipio", "timestamp", "regiao"]) \
    .filter(pre.OR(
    pre.AND(
        pre.EQ("satelite_referencia", "true"),
        pre.GE("timestamp", "2016-01-01"),
        pre.LT("timestamp", "2016-02-01"),
        pre.WITHIN(
            "POLYGON((-49.515628859948507 -19.394602563415745,-48.020567850467053 -19.610579617637825,-48.354439522883652 -21.052347219666608,-49.849500507163917 -20.836369963642884,-49.515628859948507 -19.394602563415745))"),
    ), \
    pre.NE("regiao", "SE") \
    )) \
    .max_features(10) \
    .sort_by("regiao")
    
focos = f.get()
display(focos.head())

,geometry,id,municipio,regiao,timestamp
0,POINT (-48.152 -15.939),34134137,Brasília,CO,2016-07-13T13:30:00Z
1,POINT (-47.718 -15.74),34134153,Brasília,CO,2016-07-12T17:15:00Z
2,POINT (-48.119 -15.716),34134127,Brasília,CO,2016-09-17T17:10:00Z
3,POINT (-47.955 -15.918),34134135,Brasília,CO,2016-09-17T17:10:00Z
4,POINT (-47.702 -15.737),34134145,Brasília,CO,2016-07-12T17:15:00Z
